# Network Analysis on Stack Overflow website

## 1. Introduction
Stack Overflow is a community based website that provides programmers and developers a platform to share professional knowledge. Via interactions like posting and answering questions, commenting, and etc., users participate in different topics and tend to form interests and, as a result, communities. 

This project aims to analyse the reason why users in Stack Overflow participate in certain threads. Having a deeper insight into how links formed among users and communities during time is helpful to handle possible business problems emerged in the future.

## 2. Data Source
The data used for the network analysis of Stack Overflow was acquired from the SNAP dump (https://snap.stanford.edu/data/sx-stackoverflow.html). 
SNAP dump consists of three different types of interactions: 

   + Answers to questions dataset (sx-stackoverflow-a2q);
   
   + Comments to questions dataset (sx-stackoverflow-c2q);
   
   + Comments to answers dataset (sx-stackoverflow-c2a).
   
Each dataset contains three columns:

   + ID of the source node;

   + ID of the target node;
   
   + Unix timestamp.

Overall, Stack Overflow website can be divided into two different spaces (Figure 2.1):

   + Question space. In this space users interact with each other by posting and answering questions. The link between users appear when one user answers to another user who posted the question. Answers to questions dataset was used to build the network on that space. 

   + Social space. In this space users interact with each other by commenting on each other's answers and questions. The link between users appear when one user comments another user’s answer or question. Comments to questions and Comments to answers datasets were taken to create the network on social space.
   

## 3. Idea Generation 

The main goal in network analysis of Stack Overflow is to investigate the reason why users participate in particular threads and why users can change their communities. 

It goes without saying that users participate in particular threads because of their interest in particular topics. Over time, users’ interest can change for various reasons, one of which is the influence of social space. Very often, starting the communication with several members from another threads and then increasing the number of these members can lead a user to change his previous thread into new one.

Therefore, the goal of the project can be reformulated as follows: **Can the change in social space cause the change in question space?**

The whole solution to the question can be divided into three parts:

1. Analysis of the question space: community detection (4 paragraph);

2. Analysis of the social space: triadic closures (6 and 7 paragraphs);

3. Linkage between question and social space: membership closures (6 and 8 paragraphs).


## 4. Community Detection Methods
When trying to analyse social networks, the first step is to detect the communities. Community detection mechanism took place in question space.

A community is “a group of nodes that have a higher likelihood of connecting to each other than to nodes from other communities” (Barabási, A., 2015). There have been various methods used to detect these existing communities.

To conduct network analysis two algorithms were taken into consideration:

1. The Girvan-Newman algorithm was constructed by Michelle Girvan and Mark Newman. The algorithm consists of two steps, Defining Centrality, and Hierarchical Clustering. 

+ The first step includes the definition and measuring of centrality. The algorithm needs a low centrality measure for the nodes within the same community and high measure for nodes located in different communities. To gather these measurements, the algorithm uses Link Betweenness and Random-Walk Betweenness. Link betweenness captures looks at the “role of each link in information transfer”(Barabási, A., 2015). 

+ The second step of the Girvan-Newman algorithm is the use of Hierarchical Clustering where the centrality of each link is computed, the link of the largest centrality is removed, and centrality of each link is recalculated.

2. The Louvain algorithm is an algorithm that has been helpful in community detection. It is a method that is accurate due to the fact that it focuses on networks that are located within known community structures. There are two steps to implementing the algorithm, which are repeated until the required modularity is achieved.

+ The first step includes the search for small or minute communities through the optimization of modularity in a local way.

+ The second step of the algorithm is the aggregation of nodes within the same community. 

Due to fairly large analysed network size, the choice of the community detection method was made in favour of Louvain Algorithm because  it allows to identify communities in networks with millions of nodes in a relatively short period of time in comparison to Girvan-Newman algorithm.

After applying Louvain Community Detection Algorithm, the list of nodes that changed their communities was found. Focusing only on that list, the analysis moved to the social space, where then only changes of that nodes were investigated.


In [2]:
#importing the libraries
import os
import pandas as pd
import numpy as np
import random
import networkx as nx
import matplotlib.pyplot as plt
import itertools
import community

In [3]:
def Louvain (graph):
    partition_louvain = community.best_partition(graph)
    size = float(len(set(partition_louvain.values())))
    communities_louvain = []
    for com in set(partition_louvain.values()):
        list_nodes = [nodes for nodes in partition_louvain.keys() if partition_louvain[nodes] == com]
        communities_louvain.append(list_nodes)
    return communities_louvain

In [4]:
def dictneighb (communities):
    neighb = []
    for com in communities:
        for node in com:
            dct = {}
            lst = [nd for nd in com if nd != node]
            dct[node] = lst
            neighb.append(dct)
    return neighb 

In [5]:
def community_diff (commOne, commTwo):
    difference = []
    for i in commTwo:
        dct = {}
        for j in commOne:
            node_one = list(i.keys())[0]
            node_zero = list(j.keys())[0]
            if(node_one == node_zero):
                values_zero = list(j.values())[0]
                values_one = list(i.values())[0]
                #if no intersection -> new community
                #two_communities = []
                if(bool(set(values_one).intersection(values_zero)) == False):
                    #two_communities.append(values_zero)
                    #two_communities.append(values_one)
                    dct[node_one] = values_one
                    difference.append(dct)
    return difference

## 5. Data Preparation

Data Preparation can be divided into the following steps:

1. Transform Unix timestamp data in timedelta object and extract months from that object (function timedelta()).

2. Concatenate Comment to question and Comment to answer datasets in order to build the network in social space.

3. Filter the data for Question and Social space by month column in order to analyse first 15-month time period (t0 = 14, t1 = 15).

4. Remove self-interactions for both Question and Social space (function removeSelfInter()).

5. Leave only needed columns for future analysis (src_id and tgt_id). 

In [6]:
def removeSelfInter (df):
    return df[df['src'] != df['tgt']]

In [7]:
df = pd.read_table('sx-stackoverflow-a2q.txt.gz',
                   sep=' ',
                   header=None,
                   names=['src', 'tgt', 'ts'])

In [8]:
# fix the timespan for the analysis (in days)
timespan = 30

# time elapsed since the first email
df.loc[:, 'td'] = pd.to_timedelta(df['ts'], unit='s')
df.loc[:, 'm'] = df['td'].dt.days // timespan

In [9]:
df['m'] = df['m'] - min(df['m'])
df.drop(['ts','td'],axis = 1,inplace = True)

In [112]:
df_14 = df.loc[df['m'] < 15, ['src','tgt']]
df_15 = df.loc[df['m'] < 16, ['src','tgt']]

In [113]:
#delete those users who answered on their own question by themselves
df_14 = removeSelfInter(df_14)
df_15 = removeSelfInter(df_15)

In [114]:
G_14 = nx.from_pandas_edgelist(df_14, source='src', target='tgt')
G_15 = nx.from_pandas_edgelist(df_15, source='src', target='tgt')

In [115]:
communities_14 = Louvain(G_14)
communities_15 = Louvain(G_15)

In [116]:
neighb_14 = dictneighb(communities_14)
neighb_15 = dictneighb(communities_15)

In [117]:
difference_15 = community_diff (neighb_14, neighb_15)

In [118]:
nodes_goers = []
for com in difference_15:
    nodes_goers.append(list(com.keys())[0])

In [119]:
goers_size = len(nodes_goers)

In [120]:
ComtoAns = pd.read_table('sx-stackoverflow-c2a.txt.gz',
                   sep=' ',
                   header=None,
                   names=['src', 'tgt', 'ts']) 
               
ComtoQue = pd.read_table('sx-stackoverflow-c2q.txt.gz',
                   sep=' ',
                   header=None,
                   names=['src', 'tgt', 'ts']) 

In [121]:
# fix the timespan for the analysis (in days)
timespan = 30

# time elapsed since the first comment
ComtoAns.loc[:, 'td'] = pd.to_timedelta(ComtoAns['ts'], unit='s') 
ComtoAns.loc[:, 'month'] = ComtoAns['td'].dt.days // timespan
ComtoAns['month'] = ComtoAns['month'] - ComtoAns['month'].min()
Compart1 = ComtoAns.loc[(ComtoAns['month'] < 16),['src', 'tgt','month']]


ComtoQue.loc[:, 'td'] = pd.to_timedelta(ComtoQue['ts'], unit='s') 
ComtoQue.loc[:, 'month'] = ComtoQue['td'].dt.days // timespan
ComtoQue['month'] = ComtoQue['month'] - ComtoQue['month'].min()
Compart2 = ComtoQue.loc[(ComtoQue['month'] < 16),['src', 'tgt','month']]

In [122]:
Comfull = pd.concat([Compart1,Compart2],axis=0)

Compart1 = Comfull.loc[(Comfull['month'] < 15),['src', 'tgt', 'month']]
Compart2 = Comfull.loc[(Comfull['month'] < 16) ,['src', 'tgt', 'month']]

Compart1 = removeSelfInter(Compart1)
Compart2 = removeSelfInter(Compart2)

## 6. Closures 

Assumption that changes in social space will contribute to transformations in communities was made. Thus closures detection mechanism taking place in social space is of vital importance.

Social scientists have continuously studied the links between human beings in various social settings. The use of closures has been used to explain the relationships between individuals and groups where resources and information are shared amongst people with strong relations. Scientists, such as Georg Simmel, have come up with various ideologies to explain social interactions between individuals. 

Simmel was the pioneer of the Triadic Closure. A triadic closure is the tie that occurs amongst three individuals. For example, if A-B has a relationship, and A-C has a relationship, then there is a high probability of B-C having a tie. The tie amongst these three different individuals creates what is known as a community. Simmel compared the strength of the triadic closure to that of a dyad, indicating that the removal of one individual in the closure will still lead to the existence of a tie or a community as there are two other members that are still aligned. The existence of closures are essential to understand social networks in society and can explain social interactions within communities. 

Another closure that has been frequently used in the study of social networks is membership closure. Membership closures occur when A is tied to a particular focus B, while there is a tie between A and C. The assumption is that due to the tie between A-C, C will be tied to the original focus B. The membership closure has been used to understand why individuals within the same group have similar interests in certain activities. 

The probability of appearance of the membership closure is exactly the probability of community changes. 

Therefore, the goal again can be reformulated as follows: **What is the probability of the community change according to different scenarios and factors?**

In [123]:
def find_links (node, period):
    if(period == 0):
        neighb_src = list(Compart1.loc[(Compart1['src'] == node),'tgt'].values)
        neighb_tgt = list(Compart1.loc[(Compart1['tgt'] == node),'src'].values)
        neighb = neighb_src + neighb_tgt
    else:
        neighb_src = list(Compart2.loc[(Compart2['src'] == node),'tgt'].values)
        neighb_tgt = list(Compart2.loc[(Compart2['tgt'] == node),'src'].values)
        neighb = neighb_src + neighb_tgt
    return neighb

## 7. Degrees

From the described above triadic closures, it can be concluded that if a user A formed a connection with a user B from another community who have a considered amount of neighbours from his community, the number of probable triadic closures will increase, which means that user A will have even more links with another community and highly likely will move to that community. 

The node degree is used to show how well connected a node is within its community. That is why, it was decided to take a look at nodes degrees and see what impact this factor can have. 

For the description of the idea with nodes degrees, suppose that in t0 period, node A was in community C0, while in t1 period node A moved to community C1.

The analysis based on degrees is:

1. Calculate the degree of nodes in community C1 that connected with A

2. Get the sum of degrees, that under considered assumption will have a possible impact on A’s movement from C0 to C1. 

3. Set the mean of maximum and minimum degree in the community as the boundary value.

4. Compare the degree value received in step 2 with the boundary value, to decide whether nodes are strong or weak connected to the community.

In [124]:
result = []
for com in communities_15:       
    degrees = []
    node_degrees = {}
    for node in com:
        properties = []
        node_neighbours = list(dict(G_15[node]).keys())
        node_degree = len(set(com).intersection(node_neighbours))
        properties.append(node_degree)
        node_degrees[node] = properties
        degrees.append(node_degree)
    max_degree = max(degrees)
    min_degree = min(degrees)
    boundaries_value = int((max_degree+min_degree)/2)
    for key in list(node_degrees.keys()):
        if (node_degrees[key][0]<boundaries_value):
            properties = list(node_degrees[key])
            properties.append('weak')
            properties.append(boundaries_value)
            node_degrees[key] = properties
        else:
            properties = list(node_degrees[key])
            properties.append('strong')
            properties.append(boundaries_value)
            node_degrees[key] = properties
    result.append(node_degrees)

In [125]:
degrees_class = dict()
for com in result:
    degrees_class.update(com)

## 8. Probabilities 

The final step is to calculate the probability for nodes to change their communities based on different scenarios in social space in t0 and t1 periods.

To simplify complex problem, all possible scenarios for t0 period were divided into two big cases:

1. There were no nodes in C1 connected to A in t0 period.

2. There were some links (without knowing the exact number) between A and nodes in C1 in t0.

Each of these two scenarios has two subcases in t1 period: 

1. Node A formed only one new link in C1

2. Node A formed more than one new links, regardless of the precise number, in t1 period. 

In their turn, these two subcases are considered under degree computation that separates weak and strong connections in two different possibilities. 

Eventually, eight separate cases in total were taken into consideration.

Probability is calculated by the number of eligible for each cases nodes divided by the total number of nodes that change communities. 

As a result, eight probabilities that represent eight cases were calculated.
 
With probabilities there is an ability to get some insights of how degrees, the number of former links and the number of newly appeared links influence certain nodes to change their communities.

In [126]:
option01_weak = 0
option01_strong = 0
option02_weak = 0
option02_strong = 0
option11_weak = 0
option11_strong = 0
option12_weak = 0
option12_strong = 0
for diff in difference_15:
    node_goer = list(diff.keys())[0] 
    node_goer_values = list(diff.values())[0]
    neighb_zero = find_links(node_goer,0)
    neighb_one = find_links(node_goer,1)
    if(len(neighb_zero) != 0 | len(neighb_one) != 0):
        #intersection with community in t0
        intersect_zero_com = set(neighb_zero).intersection(node_goer_values)
        intersection = set(neighb_one).intersection(node_goer_values)
        intersect_elemnum = len (intersection)
        #if there is no intersections between t0 and nodes from community
        if(bool(intersect_zero_com) == False):
            #find the intersection between t1 and nodes from community (how many links were formed:
            # we are interested in one and more than one formed links)
            if(intersect_elemnum == 1):
                intersect_elem = list(intersection)[0]
                if(degrees_class[intersect_elem][1] == 'weak'):
                    option01_weak = option01_weak + 1
                    #one new link with weak member
                else:
                    option01_strong = option01_strong + 1
                    #'one new link with strong member'
            if(intersect_elemnum > 1):
                intersect_elem = list(intersection)
                degrees = [degrees_class[element][0] for element in intersect_elem]
                summa = sum(degrees)
                if(summa < degrees_class[intersect_elem[0]][2]):
                    #several new links and weak connectivity
                    option02_weak = option02_weak + 1
                else:
                    #several new links and strong connectivity
                    option02_strong = option02_strong + 1
        #if there is intersections between t0 and nodes from community
        else:
            if (intersect_elemnum == 1):
                intersect_elem = list(intersection)[0]
                if(degrees_class[intersect_elem][1] == 'weak'):
                    #already existed several links; new formed link and weak connectivity
                    option11_weak = option11_weak + 1
                else:
                    #already existed several links; new formed link and strong connectivity
                    option11_strong = option11_strong + 1   
            if(intersect_elemnum > 1):
                #intersection with community in t1
                intersect_zero = list(intersect_zero_com)
                intersect_elem = list(intersection)
                degrees_t0 = [degrees_class[element][0] for element in intersect_zero]
                degrees_t1 = [degrees_class[element][0] for element in intersect_elem]
                summa = sum(degrees_t0)+sum(degrees_t1)
                if (summa< degrees_class[intersect_elem[0]][2]):
                    #already existed several links; several new formed link and weak connectivity
                    option12_weak = option12_strong + 1
                else:
                    #already existed several links; new formed link and strong connectivity
                    option12_weak = option12_strong + 1

In [127]:
probability_1 = option01_weak/goers_size
probability_2 = option01_strong/goers_size
probability_3 = option02_weak/goers_size
probability_4 = option02_strong/goers_size
probability_5 = option11_weak/goers_size
probability_6 = option11_strong/goers_size
probability_7 = option12_weak/goers_size
probability_8 = option12_strong/goers_size

In [128]:
print(probability_1,probability_2,probability_3,probability_4,probability_5, probability_6, probability_7, probability_8)

0.0 0.1282051282051282 0.0 0.02564102564102564 0.0 0.02564102564102564 0.02564102564102564 0.0


## 9. Key Insights  

From the received probabilities, it’s clearly seen that:

1. If no links formed in t0 period, then:

+ The more new links will be formed in t1, the higher will be the probability for a node to change its community. This can be explained by the fact that the more new neighbors from another community node will have, the higher will be the probability that this node’s interest will change to the interest of its new neighbors.  

+ The more degree the nodes from community to which the link was formed have, the higher will be the probability of community change. The reason for these rises can be explained by the probable appearance of triadic closures within social space, which implies that the node is more likely to connect to other nodes within the community via triadic closures and therefore change its community.

2.  If several links formed in t0 period, then the conclusion could be:

+ The value of probability does not drop in comparison to the first case, because in t0 period there was an initial number of neighbors from community to which node jumped and then the number of neighbors becomes bigger in t1 period (see explanations in 1.1). At the same time, the value of probability is not higher than that from the first case, which can mean that the number of links that were formed in t0 period has not a significant impact on change in communities in comparison to the number of newly created links, number of links in total and nodes’ degree that have an effect. 

To sum up, the conclusion is that changes in social space indeed can be a reason why nodes tend to change their communities.


## 10. Limitations and Further Research
In network analysis of Stack Overflow there were few limitations that are listed below:

1. The differences among probabilities get from the output are insignificant and some probabilities are equal to 0. The possible reason can be the fact of data. First 15 months were chosen for analysis when social exchanges just started and communities just appeared and there was no formed extensive interactions. Thereby, the number of nodes that changed their communities in considered time period is small, and even becomes smaller in social space because nodes that were active in question space might not participate in social space interactions.

2. Limited power of server. Due to this limitation there was no possibility to conduct the analysis on later time period (with bigger volume of data) when the interactions between users both in social and question space become more developed.

3. The reasons why users participate in certain threads must be more complex than that was explained in this work. It is impossible to consider all possible reasons, the number of which is significantly high, and interpret them well. Thus, the problem was simplified and considered under two base scenarios regardless of other possible cases (e.g. the impact of triadic closure in social space on community was not analysed in detail). 

For the future research, methods and algorithms used to analyse should be chosen carefully, taking into consideration not only speed but also accuracy and fitness. What’s more, influences of factors such as triadic closure and membership closure can be dug deeply with more sophisticated research plan.


References
==========
1. Barabási, A. (2015). Network Science. Cambridge University Press,Chapter9.
2. Easley, D. and Kleinberg, J. (2010). k Networks, Crowds, and Markets: Reasoning about a Highly Connected World. Cambridge University Press, pp.85-118.
3. FindCommunities. [online] Available at: https://sites.google.com/site/findcommunities/ 
4. Simmel, G. (1950). The Sociology of Georg Simmel. New York City: Simon and Schuster.